In [ ]:
!pip install accelerate

In [ ]:
!pip install torch torchvision torchaudio

In [ ]:
!pip install datasets optuna transformers -U colorlog evaluate huggingface_hub multiprocess xxhash regex

In [ ]:
pip install tf-keras

In [ ]:
%%bash
export TF_USE_LEGACY_KERAS=1

In [ ]:
!pip install --upgrade transformers

In [ ]:
pip install --upgrade tensorflow

In [ ]:
from datasets import load_dataset
from transformers import ViTImageProcessor, AutoModelForImageClassification, TrainingArguments, Trainer
import evaluate
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomResizedCrop,
    Resize,
    ToTensor,
)
import numpy as np
import torch
import optuna

In [ ]:
dataset = load_dataset("/home/jovyan/datafabric/fd_hf")

In [ ]:
def objective(trial):
    
    print(dataset["train"].features)
    metric = evaluate.load("accuracy")

    labels = dataset["train"].features["label"].names
    label2id, id2label = dict(), dict()
    for i, label in enumerate(labels):
        label2id[label] = i
        id2label[i] = label
        
    model_checkpoint = "google/vit-base-patch16-224"
    batch_size = 16
        
    image_processor  = ViTImageProcessor.from_pretrained(model_checkpoint)

    normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
    if "height" in image_processor.size:
        size = (image_processor.size["height"], image_processor.size["width"])
        crop_size = size
        max_size = None
    elif "shortest_edge" in image_processor.size:
        size = image_processor.size["shortest_edge"]
        crop_size = (size, size)
        max_size = image_processor.size.get("longest_edge")

    train_transforms = Compose(
            [
                RandomResizedCrop(crop_size),
                RandomHorizontalFlip(),
                ToTensor(),
                normalize,
            ]
        )

    val_transforms = Compose(
            [
                Resize(size),
                CenterCrop(crop_size),
                ToTensor(),
                normalize,
            ]
        )
    def preprocess_train(example_batch):
        """Apply train_transforms across a batch."""
        example_batch["pixel_values"] = [
            train_transforms(image.convert("RGB")) for image in example_batch["image"]
        ]
        del example_batch["image"]  # Remove the original image field
        return example_batch

    def preprocess_val(example_batch):
        """Apply val_transforms across a batch."""
        example_batch["pixel_values"] = [val_transforms(image.convert("RGB")) for image in example_batch["image"]]
        del example_batch["image"]  # Remove the original image field
        return example_batch

    train_ds = dataset['train']
    val_ds = dataset['test']

        # Apply the preprocessing
    train_ds = train_ds.with_transform(preprocess_train)
    val_ds = val_ds.with_transform(preprocess_val)
    
    model = AutoModelForImageClassification.from_pretrained("google/vit-base-patch16-224", ignore_mismatched_sizes=True, label2id=label2id,
        id2label=id2label,)

    model_name = model_checkpoint.split("/")[-1]

 
    def compute_metrics(eval_pred):
        """Computes accuracy on a batch of predictions"""
        predictions = np.argmax(eval_pred.predictions, axis=1)
        return metric.compute(predictions=predictions, references=eval_pred.label_ids)



    # Suggest hyperparameters
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-3, log=True)
    num_epochs = trial.suggest_int("num_epochs", 1, 5)
    gradient_accumulation_steps = trial.suggest_int("gradient_accumulation_steps", 1, 8)
    per_gpu_batch_size = trial.suggest_int("per_gpu_batch_size", 8, 32)
    weight_decay = trial.suggest_float("weight_decay", 0, 0.5)
    warmup_steps = trial.suggest_int("warmup_steps", 0, 100)
    # Define training arguments
    training_args = TrainingArguments(
         f"{model_name}-finetuned-forgery",
        learning_rate=learning_rate,
        num_train_epochs=num_epochs,
        gradient_accumulation_steps=gradient_accumulation_steps,
        per_gpu_train_batch_size = per_gpu_batch_size,
        weight_decay=weight_decay,
        warmup_steps=warmup_steps,
        remove_unused_columns=False,
        eval_strategy = "epoch",
        save_strategy = "epoch",
        logging_steps=10,
        load_best_model_at_end=True,
        metric_for_best_model="eval_accuracy",
        push_to_hub=False,
    )
    # Train your model
    trainer = Trainer(
        model,
        training_args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        tokenizer=image_processor,
        compute_metrics=compute_metrics,
    )
    train_results = trainer.train()
    # Evaluate your model
    eval_result = trainer.evaluate()
    
        # rest is optional but nice to have
    trainer.save_model('model_checkpoints')
    trainer.log_metrics("train", train_results.metrics)
    trainer.save_metrics("train", train_results.metrics)
    trainer.save_state()

    metrics = trainer.evaluate()
    # some nice to haves:
    trainer.log_metrics("eval", metrics)
    trainer.save_metrics("eval", metrics)
    
    return eval_result["eval_accuracy"]


In [ ]:
import mlflow
mlflow.autolog()
# Set up MLflow experiment
# mlflow.set_experiment("HF Optuna Forgery")

with mlflow.start_run(experiment_id="599198156041778570"):
    study = optuna.create_study(direction="maximize")  # maximize accuracy
    # Run the optimization
    study.optimize(objective, n_trials=1) # Adjust number of trials